In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from game import Game
from connect_net import ConnectNet
from nodes import DummyNode, Node, print_tree

import matplotlib.pyplot as plt
import numpy as np

game = Game()
net = ConnectNet()

## Game nodes

In [ ]:
model = get_model()
dataset = self_play(model, 10)

boards = []
policies = []
values = []

for board, policy, value in dataset:
    boards.append(board)
    policies.append(policy)
    values.append(value)
    
boards = np.array(boards)
policies = np.array(policies)
values = np.array(values)

model.fit(boards, [policies, values], batch_size=32, shuffle=True, epochs=10)